In [ ]:
# Place %autoreload 2 at the top of your notebook to ensure it applies to all subsequent imports.
%load_ext autoreload
%autoreload 2
 


In [57]:
from __future__ import absolute_import, division, print_function
import torch
from trainer_end_to_end import Trainer
from options import MonodepthOptions
import time
options = MonodepthOptions()
args = [
    '--data_path', '/mnt/nct-zfs/TCO-All/SharedDatasets/SCARED_Images_Resized/',
    # '--of_samples_num', '3', 
    '--batch_size', '10', 
    # '--log_frequency', '2',
    # '--log_dir', '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/debug_reloc3r_backbone/relocxr',
    '--debug',
    ]
opts = options.parse_notebook(args)# trainer.set_eval()
# opts

In [ ]:
# load model
trainer = Trainer(opts)
print('trainer initialized...')

In [ ]:
# opts.pose_model_type
trainer.opt.pose_model_type

In [ ]:
# trainer = Trainer(opts)
self = trainer #Trainer(opts)


# trainer.train()
"""Run the entire training pipeline
"""
self.epoch = 0
self.step = 0
self.start_time = time.time()
with torch.no_grad():
    # for self.epoch in range(self.opt.num_epochs):
    # self.run_epoch(self.epoch)
    """Run a single epoch of training and validation
    """
    epoch = self.epoch
    print("Training")

    self.model_optimizer_0.zero_grad()# debug only: save_mem
    self.model_optimizer.zero_grad()# debug only: save_mem

    # for batch_idx, inputs in enumerate(self.train_loader):

    train_iter = iter(self.train_loader)
    inputs = next(train_iter)
    before_op_time = time.time()

    # position
    self.set_train_0()
    if self.opt.debug:
        self.freeze_params(keys = ['position_encoder'])#debug only to save_mem
    _, losses_0 = self.process_batch_0(inputs)
    self.model_optimizer_0.zero_grad()
    # losses_0["loss"].backward()#debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train_0, max_norm=1.0)
    self.model_optimizer_0.step()

    self.set_train()
    if self.opt.debug:
        self.freeze_params(keys = ['depth_model', 'pose', 'transform_encoder'])#debug only to save_mem
    outputs, losses= self.process_batch(inputs)
    self.model_optimizer.zero_grad()
    # losses["loss"].backward() #debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train, max_norm=1.0)
    self.model_optimizer.step()

    duration = time.time() - before_op_time

    # phase = batch_idx % self.opt.log_frequency == 0

    # # if phase:
    # # self.log_time(batch_idx, duration, losses["loss"].cpu().data)
    # self.log_time(0, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, )
    # scalers_to_log = {
    #     "scalar/loss": losses["loss"],
    #     "scalar/loss_0": losses_0["loss"],
    #     # "scalar/trans_err": metric_errs["trans_err"].mean(),
    #     # "scalar/rot_err": metric_errs["rot_err"].mean()
    # }
    # for k, v in errs.items():
    #     print(f"scalar/{k}: {v}")
    #     scalers_to_log[f"scalar/{k}"] = v
    # concat_img, img_order_strs = self.log("train", inputs, outputs, scalers_to_log, compute_vis=True, online_vis=False)
    
    
    # Use the accumulated loss for logging (multiply back to show effective loss)
    # effective_loss = losses["loss"] * (self.opt.accumulate_steps / max(1, accumulate_step % self.opt.accumulate_steps))
    # self.log_time(batch_idx, duration, effective_loss.cpu().data)
    losses_to_log = {
        "loss": losses["loss"],
        "loss_0": losses_0["loss"],
    }
    errs_to_log = {}
    errs = self.compute_pose_metrics(inputs, outputs)
    for k, v in errs.items():
        assert len(v) == len(self.opt.frame_ids)-1, f'{k}: {v}'
        # v is a list of torch tensors, average over them
        errs_to_log[f"{k}"] = torch.mean(torch.stack(v)).item()

    self.log_time(self.step, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, 
                    errs_to_log)
    scalers_to_log = {**losses_to_log, **errs_to_log}
    scalers_to_log = {f"scalar/{k}": v for k, v in scalers_to_log.items()}

    concat_img, img_order_strs = self.log("train", inputs, outputs, 
                scalers_to_log, 
                compute_vis=True)
    
    
    
    
    # #compute_pose_err
    for k, v in errs.items():
        print(f"{k}: {v}")

    # print('trans_err:', outputs[("trans_err", 0)])
    # print('rot_err:', outputs[("rot_err", 0)])

 


    import os, cv2
    save_path = "/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/"
    # remove the first char before the first _
    # img_order_strs = img_order_strs.split('_')[1:]
    # img_order_strs = '_'.join(img_order_strs)
    save_path = os.path.join(save_path, f"{img_order_strs}.png")
    cv2.imwrite(save_path, concat_img)
    print(f"saved train_concat_img.png in {save_path}")

    self.step += 1
    
    # break
        
    self.model_lr_scheduler.step()
    self.model_lr_scheduler_0.step()

    # if (self.epoch + 1) % self.opt.save_frequency == 0:
    #     self.save_model()
    # break
    


In [61]:

# self.set_eval()
# try:
#     # inputs = self.val_iter.next()
#     inputs = next(self.val_iter)
# except StopIteration:
#     self.val_iter = iter(self.val_loader)
#     inputs = self.val_iter.next()

# with torch.no_grad():
#     # outputs, losses = self.process_batch_val(inputs)
#     """Pass a minibatch through the network and generate images and losses
#     """
#     for key, ipt in inputs.items():
#         inputs[key] = ipt.to(self.device)

#     if self.opt.pose_model_type == "shared":
#         # If we are using a shared encoder for both depth and pose (as advocated
#         # in monodepthv1), then all images are fed separately through the depth encoder.
#         print('CHECK: self.opt.pose_model_type == "shared"')
#         all_color_aug = torch.cat([inputs[("color_aug", i, 0)] for i in self.opt.frame_ids])
#         all_features = self.models["encoder"](all_color_aug)
#         all_features = [torch.split(f, self.opt.batch_size) for f in all_features]

#         features = {}
#         for i, k in enumerate(self.opt.frame_ids):
#             features[k] = [f[i] for f in all_features]

#         outputs = self.models["depth"](features[0])
#     else:
#         print('CHECK: self.opt.pose_model_type != "shared"')
#         # Otherwise, we only feed the image with frame_id 0 through the depth encoder
#         features = self.models["encoder"](inputs["color_aug", 0, 0])
#         outputs = self.models["depth"](features)

#     if self.opt.predictive_mask:
#         outputs["predictive_mask"] = self.models["predictive_mask"](features)

#     if self.use_pose_net:
#         outputs.update(self.predict_poses(inputs, features, outputs))

#     self.generate_images_pred(inputs, outputs)
#     losses = self.compute_losses_val(inputs, outputs)

#     # return outputs, losses

#     self.log("val", inputs, outputs, losses)
#     del inputs, outputs, losses